In [1]:
import os
# Create save dir
save_dir = "C:/Users/Usuario/Desktop/imperial_projects/VSCode/pcgym2/paper_base_opt_train_eval/ppo"
os.makedirs(save_dir, exist_ok=True)

In [2]:
!dir pc-gym

 Volume in drive C is Windows 
 Volume Serial Number is F0B8-32A3

 Directory of c:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\pc-gym

15/08/2024  13:03    <DIR>          .
15/08/2024  15:19    <DIR>          ..
15/08/2024  13:03                25 .gitattributes
15/08/2024  13:03    <DIR>          .github
15/08/2024  13:03             2,940 .gitignore
15/08/2024  13:03    <DIR>          .vscode
15/08/2024  13:03    <DIR>          Control-Club-Challenge
15/08/2024  13:03    <DIR>          docs
15/08/2024  13:03               188 environment.yml
15/08/2024  13:03    <DIR>          example_notebooks
15/08/2024  13:03             1,094 LICENSE
15/08/2024  13:03             3,835 mkdocs.yml
15/08/2024  13:03    <DIR>          pc-gym_paper
15/08/2024  13:03             1,309 pyproject.toml
15/08/2024  13:03           132,281 Quick_Start.ipynb
15/08/2024  13:03             3,970 README.md
15/08/2024  13:03                95 requirements.txt
15/08/2024  13:07    <DIR>          src
1

In [3]:
%cd C:/Users/Usuario/Desktop/imperial_projects/VSCode/pcgym2/pc-gym

C:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\pc-gym


c:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\new_venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from dataclasses import dataclass
import numpy as np
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
import torch.nn.functional as F
import copy
from stable_baselines3 import PPO,SAC,DDPG,TD3
import pcgym
from pcgym import make_env
import jax.numpy as jnp
#Global params
T = 26
nsteps = 101
# Global seed for reproducibility
seed = 1990

c:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\new_venv\Lib\site-packages\do_mpc\sysid\__init__.py:15: UserWarning: The ONNX feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The ONNX feature is not available. Please install the full version of do-mpc to access this feature.')
c:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\new_venv\Lib\site-packages\do_mpc\opcua\__init__.py:14: UserWarning: The opcua feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The opcua feature is not available. Please install the full version of do-mpc to access this feature.')


In [5]:
import wandb
import os
import sys
from stable_baselines3 import PPO
import torch as th
import numpy as np
from stable_baselines3.common.vec_env import DummyVecEnv

##################################################################################
# Environment and RL Definition
##################################################################################

# Enter required setpoints for each state. Enter None for states without setpoints.
SP = {
    'T': [324.475443431599 for _ in range(5)] + [340.0 for _ in range(nsteps - 5)],
}

# Continuous box action space
action_space = {
    'low': np.array([250]),
    'high': np.array([350])
}

# Continuous box observation space ([CA, T, CA_Setpoint, T_Setpoint])
observation_space = {
    'low': np.array([0.0, 200, 300]),
    'high': np.array([1, 600, 400])
}

r_scale = {
    'T': 1e-6  # Reward scale for each state,
}

# Define disturbance bounds
disturbance_bounds = {
    'low': np.array([310]),
    'high': np.array([390])
}

# Environment parameters
env_params_template = {
    'Nx': 2,
    'N': 100,
    'tsim': 26,
    'Nu': 1,
    'SP': SP,
    'o_space': observation_space,
    'a_space': action_space,
    'x0': np.array([0.87725294608097, 324.475443431599, 324.475443431599]),
    'model': 'cstr',
    'r_scale': r_scale,
    'normalise_a': True,
    'normalise_o': True,
    'noise': True,
    'integration_method': 'casadi',
    'noise_percentage': 0.001,  # 0.001,
    'disturbance_bounds': disturbance_bounds
}

# Seed everything for reproducibility
def set_global_seeds(seed):
    np.random.seed(seed)
    th.manual_seed(seed)
    if th.cuda.is_available():
        th.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# Function to create random disturbances
def create_random_disturbances(seed, nsteps, low=315, high=385):
    # Set the global seed for reproducibility
    set_global_seeds(seed)
    values = np.random.uniform(low, high, 3)  # Generate three random disturbance values within the specified range
    times = np.sort(np.random.choice(range(1, nsteps - 1), 2, replace=False))  # Select two unique time steps for disturbances
    times = np.append(times, nsteps)  # Append the total number of steps to get three periods
    times = np.diff([0] + times.tolist())  # Calculate the duration of each disturbance period
    disturbances = {'Ti': np.repeat(values, times)}  # Repeat the disturbance values according to the calculated durations
    return disturbances

# Create multiple environments with different disturbances
def create_parallel_envs(n_envs, seed):
    set_global_seeds(seed)
    envs = []
    disturbances_list = []
    for i in range(n_envs):
        env_params = env_params_template.copy()
        disturbances = create_random_disturbances(seed + i, nsteps)
        env_params.update({'disturbances': disturbances})
        disturbances_list.append(disturbances)
        envs.append(lambda: make_env(env_params))
    return DummyVecEnv(envs), disturbances_list


# Create evaluation environment using DummyVecEnv
def create_eval_env(seed, n_envs=1):
    set_global_seeds(seed)
    envs = []
    for i in range(n_envs):
        env_params = env_params_template.copy()
        disturbances = create_random_disturbances(seed + i, nsteps)
        env_params.update({'disturbances': disturbances})
        envs.append(lambda: make_env(env_params))
    return DummyVecEnv(envs)

config = {
    "seed": 1990,
}

eval_env = create_eval_env(seed=config['seed'], n_envs=1)

# Set the global seed for reproducibility
set_global_seeds(config['seed'])

# Number of parallel environments
n_envs = 5

# Function to create an environment
def create_env(env_params):
    set_global_seeds(seed)
    env = make_env(env_params)
    return env

# Function to create an environment with specific disturbances
def create_eval_env_with_disturbances(seed, disturbances):
    set_global_seeds(seed)
    env_params = env_params_template.copy()
    env_params.update({
        'disturbances': disturbances,
        'disturbance_bounds': disturbance_space,
        'N': 101,
    })
    return create_env(env_params)

# Disturbance bounds for 'Ti'
disturbance_space = {
    'low': np.array([310]),
    'high': np.array([390])
}

# Create the first disturbances
first_disturbance = create_random_disturbances(seed, nsteps)

# Print the first disturbance for comparison
print("First disturbance used in evaluation:", first_disturbance)

# Create environments with the first disturbance
env_first = create_eval_env_with_disturbances(seed, first_disturbance)

# Number of rollouts for evaluation
rollout_number = 10

First disturbance used in evaluation: {'Ti': array([365.53840802, 365.53840802, 365.53840802, 365.53840802,
       365.53840802, 365.53840802, 365.53840802, 365.53840802,
       365.53840802, 365.53840802, 365.53840802, 365.53840802,
       365.53840802, 365.53840802, 365.53840802, 365.53840802,
       365.53840802, 320.42850404, 320.42850404, 320.42850404,
       320.42850404, 320.42850404, 320.42850404, 320.42850404,
       320.42850404, 320.42850404, 320.42850404, 320.42850404,
       320.42850404, 320.42850404, 320.42850404, 320.42850404,
       320.42850404, 320.42850404, 320.42850404, 320.42850404,
       320.42850404, 320.42850404, 320.42850404, 320.42850404,
       320.42850404, 320.42850404, 320.42850404, 320.42850404,
       320.42850404, 320.42850404, 320.42850404, 320.42850404,
       320.42850404, 320.42850404, 320.42850404, 320.42850404,
       321.83357664, 321.83357664, 321.83357664, 321.83357664,
       321.83357664, 321.83357664, 321.83357664, 321.83357664,
       321

In [6]:
import optuna
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from datetime import datetime
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_contour

class OptimizationStudy:
    def __init__(self, env, best_model_path, save_dir, study_name_prefix="optimization", bounds=None) -> None:
        self.env = env
        try:
            self.PPO_model = PPO.load(best_model_path)
        except Exception as e:
            print(f"Error loading PPO model from {best_model_path}: {e}")
            raise

        self.save_dir = save_dir
        self.study_name_prefix = study_name_prefix
        self.study_name = self._generate_study_name()
        self.study = None
        self.bounds = bounds if bounds is not None else {
            'N': (1, 30),
            'R': (1e-10, 1),
            'Q': (1e-10, 1)
        }

    def _generate_study_name(self):
        current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
        return f"{self.study_name_prefix}_{current_time}"

    def _get_original_env(self):
        """Helper method to access the correct environment depending on the type of env."""
        if hasattr(self.env, 'envs'):  # If it's a DummyVecEnv, access the first environment
            return self.env.envs[0]
        else:
            return self.env  # Otherwise, just return the environment itself

    def objective(self, trial):
        try:
            # Suggest values for N, R, and Q based on the given bounds
            N = trial.suggest_int('N', self.bounds['N'][0], self.bounds['N'][1])
            R_value = trial.suggest_float('R', self.bounds['R'][0], self.bounds['R'][1])
            Q_value = trial.suggest_float('Q', self.bounds['Q'][0], self.bounds['Q'][1])

            # Extract parameters for MPC from env_params_template
            Nu = env_params_template['Nu']
            Nx_oracle = env_params_template['Nx']

            # Define R as a diagonal matrix with R_value on the diagonal
            R = np.diag([R_value] * Nu)

            # Define Q as an identity matrix scaled by Q_value
            Q = np.eye(Nx_oracle) * Q_value

            # Update MPC parameters with the new values
            MPC_params = {'N': N, 'R': R, 'Q': Q}

            # Debugging info
            print(f"Evaluating with N={N}, R_value={R_value}, Q_value={Q_value}")

            # Access the original environment
            original_env = self._get_original_env()

            # Get rollouts with the environment using the PPO model and the specified MPC parameters
            evaluator, data = original_env.get_rollouts({'PPO': self.PPO_model},
                                                        reps=10, oracle=True, MPC_params=MPC_params)

            # Aggregate the performance values across the rollouts
            performance_values = data['oracle']["r"].sum(axis=1)  # This gives an array of performance values
            performance = performance_values.mean()  # Take the mean (or you could use sum, max, etc.)

            return performance
        except Exception as e:
            print(f"Error during trial execution: {e}")
            raise

    def run_optimization(self, n_trials: int):
        try:
            self.study = optuna.create_study(direction='maximize')
            self.study.optimize(self.objective, n_trials=n_trials)  # Removed n_jobs for Jupyter
        except Exception as e:
            print(f"Error during optimization: {e}")
            raise

    def save_results(self):
        try:
            # Save the study results to a text file
            result_file = f"{self.save_dir}/optimization_results_{self.study_name}.txt"
            with open(result_file, 'w') as f:
                f.write(f"Study name: {self.study_name}\n")
                f.write(f"Best parameters:\n")
                f.write(f"N = {self.study.best_params['N']}\n")
                f.write(f"R = {self.study.best_params['R']:.9e}\n")
                f.write(f"Q = {self.study.best_params['Q']:.9e}\n")
                f.write(f"Best performance: {self.study.best_value:.9f}\n")
        except Exception as e:
            print(f"Error saving results: {e}")
            raise

    def print_results(self):
        try:
            print(f'Study name: {self.study_name}')
            print('Best parameters:')
            print(f"N = {self.study.best_params['N']}")
            print(f"R = {self.study.best_params['R']:.6e}")
            print(f"Q = {self.study.best_params['Q']:.6e}")
            print(f"Best performance: {self.study.best_value:.6f}")
        except Exception as e:
            print(f"Error printing results: {e}")
            raise

    def visualize_results(self):
        try:
            plot_optimization_history(self.study)
            plt.show()
            plot_param_importances(self.study)
            plt.show()
            plot_contour(self.study, params=['N', 'R', 'Q'])
            plt.show()
        except Exception as e:
            print(f"Error visualizing results: {e}")
            raise

    def get_best_params(self):
        try:
            return {
                'N': self.study.best_params['N'],
                'R': np.diag([self.study.best_params['R']] * env_params_template['Nu']),
                'Q': np.eye(env_params_template['Nx']) * self.study.best_params['Q']
            }
        except Exception as e:
            print(f"Error getting best parameters: {e}")
            raise


c:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\new_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Usage example in Jupyter notebook:
save_dir = "C:/Users/Usuario/Desktop/imperial_projects/VSCode/pcgym2/paper_base_opt_train_eval/ppo"
best_model_path = f"{save_dir}/best_model.zip"
optimization = OptimizationStudy(eval_env, best_model_path, save_dir)
optimization.run_optimization(100)
optimization.print_results()
optimization.save_results()  # Save results to a file
optimization.visualize_results()
best_params = optimization.get_best_params()

c:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\new_venv\Lib\site-packages\stable_baselines3\common\save_util.py:449: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  

Evaluating with N=21, R_value=0.7801443243516485, Q_value=0.8341092379317139


[I 2024-08-15 15:21:36,699] Trial 0 finished with value: -0.40494931772628745 and parameters: {'N': 21, 'R': 0.7801443243516485, 'Q': 0.8341092379317139}. Best is trial 0 with value: -0.40494931772628745.


Evaluating with N=15, R_value=0.5981789145020329, Q_value=0.6755068762210144


[I 2024-08-15 15:21:45,305] Trial 1 finished with value: -0.4049493177262836 and parameters: {'N': 15, 'R': 0.5981789145020329, 'Q': 0.6755068762210144}. Best is trial 1 with value: -0.4049493177262836.


...

Evaluating with N=1, R_value=0.6026814866558521, Q_value=0.32528386011266824


[I 2024-08-15 15:22:39,156] Trial 8 finished with value: -0.40494931772632503 and parameters: {'N': 1, 'R': 0.6026814866558521, 'Q': 0.32528386011266824}. Best is trial 6 with value: -0.4049493177262645.


Evaluating with N=11, R_value=0.04982330631382471, Q_value=0.39970533038087813


[I 2024-08-15 15:22:45,616] Trial 9 finished with value: -0.2267469200372712 and parameters: {'N': 11, 'R': 0.04982330631382471, 'Q': 0.39970533038087813}. Best is trial 9 with value: -0.2267469200372712.


Study name: optimization_20240815_152127
Best parameters:
N = 11
R = 4.982331e-02
Q = 3.997053e-01
Best performance: -0.226747


In [1]:
# Set the global seed for reproducibility
set_global_seeds(seed)

# Define a new disturbance pattern that is out of the scope trained
new_disturbance = {'Ti': np.repeat([390, 370, 310], [nsteps//4, nsteps//2, nsteps//4])}

# Create a new environment with the custom disturbance
env_new = create_eval_env_with_disturbances(seed, new_disturbance)

# Define the OptimizationStudy class with this new environment
optimization_new = OptimizationStudy(env_new, best_model_path, save_dir)

# Run the optimization process with the new disturbance
optimization_new.run_optimization(100)

# Print and save the results
optimization_new.print_results()
optimization_new.save_results()

# Visualize the optimization results
optimization_new.visualize_results()

# Get the optimized parameters
best_params_new = optimization_new.get_best_params()

print(f"Optimized Parameters with New Disturbance: {best_params_new}")

NameError: name 'set_global_seeds' is not defined